In [5]:
import torch
import time
import numpy as np


def benchmark(force_cpu=False):
    # Выбор устройства
    if force_cpu:
        device = torch.device('cpu')
        print("Принудительно используется CPU")
    else:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    sequence_length = 50
    # Меньше тестов для CPU (они намного медленнее)
    num_trials = 10 if device.type == 'cpu' else 1000
    
    # Информация об устройстве
    if device.type == 'cuda':
        gpu_name = torch.cuda.get_device_name(0)
        gpu_memory = torch.cuda.get_device_properties(0).total_memory / (1024**3) # GB
        device_info = f"GPU: {gpu_name} ({gpu_memory:.1f} GB)"
    else:
        try:
            import psutil
            cpu_cores = psutil.cpu_count()
            device_info = f"CPU: {cpu_cores} cores"
        except:
            device_info = "CPU"
    
    print(f"Устройство: {device_info}")
    print("Загрузка модели.")
    model = YOLO_LSTM(
        yolo_ckpt="/workspace/yolo11m.pt",
        hidden_dim=256,
        num_layers=1,
        sequence_length=50
    )
    
    model = model.to(device)
    model.eval()
    
    # Подготовка данных
    batch_input = torch.randn(1, sequence_length, 3, 256, 256, device=device)
    
    # Прогрев зависит от устройства
    if device.type == 'cuda':
        print("Прогрев GPU.")
        warmup_iterations = 10
    else:
        print("Минимальная инициализация CPU.")
        warmup_iterations = 2
    
    with torch.no_grad():
        for _ in range(warmup_iterations):
            _ = model(batch_input)
    
    # Синхронизация после прогрева
    if device.type == 'cuda':
        torch.cuda.synchronize()
    
    # Измерения
    print(f"Измерение производительности ({num_trials} тестов)...")
    times = []

    # Отключаем автоград
    with torch.no_grad():
        for i in range(num_trials):
            # Показываем прогресс для CPU (тесты медленные)
            if device.type == 'cpu':
                print(f"  Тест {i+1}/{num_trials}...")
            
            # Синхронизация с GPU перед началом
            if device.type == 'cuda':
                torch.cuda.synchronize()
            
            start_time = time.perf_counter()
            output = model(batch_input)
            
            # Синхронизация с GPU после завершения
            if device.type == 'cuda':
                torch.cuda.synchronize()
            
            end_time = time.perf_counter()
            
            inference_time = (end_time - start_time) * 1000
            times.append(inference_time)
            
            # Показываем время для CPU
            if device.type == 'cpu':
                print(f"    Время: {inference_time:.1f} мс")
    
    # Основные метрики
    mean_time = np.mean(times)
    std_time = np.std(times)
    sequence_fps = 1000 / mean_time
    frame_throughput = sequence_length / (mean_time / 1000)
    
    return {
        'device_info': device_info,
        'latency_ms': mean_time,
        'sequence_fps': sequence_fps,
        'throughput_fps': frame_throughput
    }


def verify_measurement_correctness():
    """Проверка корректности измерений"""
    print("\n1. torch.no_grad() - отключение автограда")
    print("2. model.eval() - отключение dropout/batchnorm")  
    print("3. torch.cuda.synchronize() - синхронизация GPU")
    print("4. Прогрев GPU перед измерениями")
    print("5. Множественные измерения для статистики")
    print("6. perf_counter() - точное измерение времени")


if __name__ == "__main__":
    results_gpu = None
    results_cpu = None
    
    
    if torch.cuda.is_available():
        print("\nPhase 1: GPU Оценка производительности")
        print("-" * 40)
        results_gpu = benchmark(force_cpu=False)
        
        print(f"\nGPU Результаты:")
        print(f"  Устройство: {results_gpu['device_info']}")
        print(f"  Inference Latency: {results_gpu['latency_ms']:.1f} ms")
        print(f"  Sequence Processing Rate: {results_gpu['sequence_fps']:.1f} sequences/sec")
        print(f"  Frame Throughput: {results_gpu['throughput_fps']:.0f} frames/sec")
    else:
        print("\nGPU недоступен.")
    
    print(f"\nPhase 2: CPU Оценка производительности")
    print("-" * 40)
    print("NOTE: CPU benchmark может занять несколько минут.")
    
    user_input = input("Продолжить с CPU? (y/n): ")
    if user_input.lower() == 'y':
        results_cpu = benchmark(force_cpu=True)
        
        print(f"\nCPU Результаты:")
        print(f"  Устройство: {results_cpu['device_info']}")
        print(f"  Inference Latency: {results_cpu['latency_ms']:.1f} ms")
        print(f"  Sequence Processing Rate: {results_cpu['sequence_fps']:.1f} sequences/sec")
        print(f"  Frame Throughput: {results_cpu['throughput_fps']:.0f} frames/sec")
    else:
        print("CPU benchmark пропущен.")
    
    print(f"\n" + "=" * 70)
    print("Сравнение производительности.")
    print("=" * 70)
    
    if results_gpu and results_cpu:
        
        print(f"{'Метрики':<25} {'GPU':<15} {'CPU':<15}")
        print("-" * 70)
        print(f"{'Inference Latency (ms)':<25} {results_gpu['latency_ms']:<15.1f} {results_cpu['latency_ms']:<15.1f}")
        print(f"{'Sequence FPS':<25} {results_gpu['sequence_fps']:<15.1f} {results_cpu['sequence_fps']:<15.1f}")
        print(f"{'Frame Throughput':<25} {results_gpu['throughput_fps']:<15.0f} {results_cpu['throughput_fps']:<15.0f}")
    
    verify_measurement_correctness() 


Phase 1: GPU Оценка производительности
----------------------------------------
Устройство: GPU: NVIDIA GeForce RTX 5090 (31.4 GB)
Загрузка модели.
Прогрев GPU.
Измерение производительности (1000 тестов)...

GPU Результаты:
  Устройство: GPU: NVIDIA GeForce RTX 5090 (31.4 GB)
  Inference Latency: 15.4 ms
  Sequence Processing Rate: 65.0 sequences/sec
  Frame Throughput: 3248 frames/sec

Phase 2: CPU Оценка производительности
----------------------------------------
NOTE: CPU benchmark может занять несколько минут.


Продолжить с CPU? (y/n):  y


Принудительно используется CPU
Устройство: CPU: 64 cores
Загрузка модели.
Минимальная инициализация CPU.
Измерение производительности (10 тестов)...
  Тест 1/10...
    Время: 1296.6 мс
  Тест 2/10...
    Время: 1260.2 мс
  Тест 3/10...
    Время: 1231.1 мс
  Тест 4/10...
    Время: 1250.7 мс
  Тест 5/10...
    Время: 1240.5 мс
  Тест 6/10...
    Время: 1311.0 мс
  Тест 7/10...
    Время: 1284.5 мс
  Тест 8/10...
    Время: 1261.2 мс
  Тест 9/10...
    Время: 1204.3 мс
  Тест 10/10...
    Время: 1209.8 мс

CPU Результаты:
  Устройство: CPU: 64 cores
  Inference Latency: 1255.0 ms
  Sequence Processing Rate: 0.8 sequences/sec
  Frame Throughput: 40 frames/sec

Сравнение производительности.
Метрики                   GPU             CPU            
----------------------------------------------------------------------
Inference Latency (ms)    15.4            1255.0         
Sequence FPS              65.0            0.8            
Frame Throughput          3248            40             
